In [1]:
# Put all the imports here
import folium
import numpy as np
import pandas as pd
import plotly as ply
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

from geopy.geocoders import Nominatim
from matplotlib import pyplot as plt

# Make plotly work in offline mode
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [18]:
h1b_frame = pd.read_csv('./data/h1b.csv') # dataset to large, please download from README.md

In [3]:
h1b_frame_with_states = pd.read_csv("./data/h1b_with_states.csv")

In [99]:
cap_exampt_employee_list = pd.read_csv("./data/cap-exampt-employee-list.csv", encoding="ISO-8859-1")
cap_exampt_employee_list = list(cap_exampt_employee_list['employeer'])
cap_exampt_employee_list.append('university')
cap_exampt_employee_list.append('college')

In [117]:
years = [2011, 2012, 2013, 2014, 2015, 2016]
cap_exempt_counts = []

for y in years: 
    h1b_frame_group = h1b_frame_with_states.loc[ h1b_frame_with_states['CASE_STATUS'] == 'CERTIFIED']
    h1b_frame_group = h1b_frame_group.loc[ h1b_frame_group['state'] != 'Na']
    h1b_frame_group = h1b_frame_group.loc[ h1b_frame_group['YEAR'] == y ]
    h1b_frame_group_exempt = h1b_frame_group.loc[ h1b_frame_group['EMPLOYER_NAME'].str.contains('|'.join(cap_exampt_employee_list), case=False, na=False, regex=True) ]
    cap_exempt = h1b_frame_group_exempt.count()['Unnamed: 0']
    cap_exempt_counts.append(cap_exempt)

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



In [122]:
years = [2011, 2012, 2013, 2014, 2015, 2016]
total_counts = []

for y in years: 
    h1b_frame_group = h1b_frame_with_states.loc[ h1b_frame_with_states['CASE_STATUS'] == 'CERTIFIED']
    h1b_frame_group = h1b_frame_group.loc[ h1b_frame_group['state'] != 'Na']
    h1b_frame_group = h1b_frame_group.loc[ h1b_frame_group['YEAR'] == y ]
    total_count = h1b_frame_group.count()['Unnamed: 0']
    total_counts.append(total_count)

In [127]:
# copying result so it is easier to use later
years = [2011, 2012, 2013, 2014, 2015, 2016]
cap_exempt_counts = [25475, 24167, 23244, 23873, 24683, 24589]
total_counts = [307581, 352169, 382590, 454738, 546705, 569180]
non_cap_exempt = [282106, 328002, 359346, 430865, 522022, 544591]

In [183]:
labels = ['Cap Exempt', 'Cap Non-exempt']

for i in range(0, len(cap_exempt_counts)):
    layout = {
        "title" : ("Cap Exempt vs Cap Non-Exempt in %s" % years[i]),
    }
    
    data = {
        "values":[cap_exempt_counts[i], non_cap_exempt[i]],
        "labels":labels,
        "name": years[i],
        "type": "pie",
    }

    fig = go.Figure(
        data=[data], 
        layout=layout
    )
    
    iplot(fig, filename='donut')

In [ ]:
# Potentially want to apply for H1B? What are the companies with top salaries, most H1B applications, etc in Bay Area

In [269]:
h1b_frame_cali = h1b_frame_with_states.loc[ h1b_frame_with_states['CASE_STATUS'] == 'CERTIFIED']
h1b_frame_cali = h1b_frame_cali.loc[ h1b_frame_cali['state'] == 'California']
h1b_frame_cali = h1b_frame_cali.loc[ h1b_frame_cali['YEAR'] == 2016]

In [270]:
categories = ['computer']
h1b_frame_cali = h1b_frame_cali.loc[ h1b_frame_cali['SOC_NAME'].str.
                                    contains('|'.join(categories), case=False, na=False, regex=True) ]


In [273]:
lat = [36.5, 41.5] # San francisco, Okland, WC etc
lon = [-123.0, -120.0] # from Santa Rosa to Gillroy

h1b_frame_cali = h1b_frame_cali.loc[ 
    (h1b_frame_cali['lat'].between(lat[0], lat[1], inclusive=True)) &
    (h1b_frame_cali['lon'].between(lon[0], lon[1], inclusive=True))
]

In [283]:
h1b_frame_cali.groupby('EMPLOYER_NAME').count().sort_values('Unnamed: 0', ascending=False)

,Unnamed: 0,Unnamed: 0.1,CASE_STATUS,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat,state
EMPLOYER_NAME,,,,,,,,,,,,
INFOSYS LIMITED,2582,2582,2582,2582,2582,2582,2582,2582,2582,2582,2582,2582
WIPRO LIMITED,1251,1251,1251,1251,1251,1251,1251,1251,1251,1251,1251,1251
"HCL AMERICA, INC.",561,561,561,561,561,561,561,561,561,561,561,561
TATA CONSULTANCY SERVICES LIMITED,541,541,541,541,541,541,541,541,541,541,541,541
"TECH MAHINDRA (AMERICAS),INC.",538,538,538,538,538,538,538,538,538,538,538,538
ACCENTURE LLP,528,528,528,528,528,528,528,528,528,528,528,528
IBM INDIA PRIVATE LIMITED,482,482,482,482,482,482,482,482,482,482,482,482
INTEL CORPORATION,381,381,381,381,381,381,381,381,381,381,381,381
APPLE INC.,363,363,363,363,363,363,363,363,363,363,363,363
